### Memory distortions in the hybrid model

#### Installation:

Tested with tensorflow 2.11.0 and Python 3.10.9.

In [ ]:
!pip install -r requirements.txt
!pip install importlib-metadata==4.13.0

#### Imports and set up:

In [ ]:
from end_to_end import *
from extended_model import *
from extended_model_utils import *
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import pandas 
import pickle
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

dims = (64,64,3)

#### Prepare filtered dataframe for Carmichael simulations

In [ ]:
dataset = 'shapes3d'
ds = tfds.load(dataset, split='train', shuffle_files=False, data_dir='./data/')
ds_info = tfds.builder(dataset).info
df = tfds.as_dataframe(ds, ds_info)

filtered_df = df[
    (df['label_floor_hue'] == 1) &
    (df['label_object_hue'] == 2) &
    (df['label_orientation'] == 7) &
    (df['label_scale'] == 0) &
    (df['label_wall_hue'] == 3)
]

filtered_df['image'] = filtered_df['image']/255

with open('subset.pickle', 'wb') as handle:
    pickle.dump(filtered_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Load previously trained VAE

In [ ]:
# set tensorflow and numpy random seeds to make outputs reproducible
tf.random.set_seed(321)
np.random.seed(321)

# set tensorflow image format to arrays of dim (width, height, num_channels)
tf.keras.backend.set_image_data_format('channels_last')

# set number of epochs (actually the max number as early stopping is enabled)
generative_epochs = 50
# set number of images
num_ims = 10000

# dataset specific parameters:
# the extended model is tested with mnist only
# a lower value of beta is set for the larger image datasets to avoid overflow errors
# a higher latent dimension is set for the larger images to ensure good reconstruction
# note that a larger VAE is used for shapes3d and symmetric_solids than for mnist and fashion_mnist
params = {'extended': True,
          'hopfield_beta': 5,
          'latent_dim': 20}

net, vae = run_end_to_end(dataset='shapes3d', 
                          generative_epochs=generative_epochs, 
                          num=num_ims, 
                          latent_dim=params['latent_dim'], 
                          interpolate=False,
                          few_shot=False,
                          do_vector_arithmetic=False,
                          kl_weighting=1,
                          lr = 0.001,
                          hopfield_beta=params['hopfield_beta'])

#### Testing distortions at different thresholds

In [ ]:
thresholds = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
errors = []
counts = []

for t in thresholds:
    err, pixel_count = test_extended_model(vae=vae, dataset=dataset, threshold=t,
                                           latent_dim=params['latent_dim'], 
                                           return_errors_and_counts=True, beta=2)
    errors.append(err)
    counts.append(pixel_count)


In [ ]:
def plot_distortions(thresholds, errors, counts):
    fig, ax = plt.subplots(figsize=(4,3))
    ax2 = ax.twinx()

    ax.set_ylabel("Reconstruction error", color='red')
    ax2.set_ylabel('No. of sensory feature units', color='blue')

    ax.plot(thresholds, errors, color='red')
    ax2.plot(thresholds, counts, color='blue')

    plt.title('Reconstruction error and memory \ndimension at different thresholds')

    ax.set_xlabel('Threshold')
    plt.savefig(f'hybrid_model/error_thresholds_{dataset}.png', bbox_inches = "tight")
    return fig

fig = plot_distortions(thresholds, errors, counts)

#### Simulating semantic distortions of episodic memory

In [ ]:
with open('subset.pickle', 'rb') as handle:
    filtered_df = pickle.load(handle)

filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head()

In [ ]:
attributes_to_match = ['label_floor_hue', 'label_object_hue', 'label_orientation',
                       'label_scale', 'label_wall_hue']

def get_diff_shape_im(df, i, shape_id):
    attribute_values = df.iloc[i][attributes_to_match]
    filtered_df = df[(df[attributes_to_match] == attribute_values).all(axis=1)]
    new_im = filtered_df.loc[filtered_df['label_shape'] == shape_id, 'image'].iloc[0]
    return new_im

In [ ]:
def test_extended_model_carmichael(df, dataset='shapes3d', vae=None, beta=5, generative_epochs=100, num_ims=1000, latent_dim=10,
                        threshold=0.01, shape_id=0):
    dims = dims_dict[dataset]

    # Split the data_indices instead of the data and labels directly
    test_data = df['image']
    
    # create variant of this dataset with unusual features
    test_data_squares = [add_white_square(d, dims) for d in test_data][0:4]
    test_data_squares = np.stack(test_data_squares, axis=0).astype("float32")[0:4]

    # get reconstructed images and predicted labels for test data
    predictions = get_predictions(test_data_squares, vae)
    diff = get_true_pred_diff(test_data_squares, predictions)

    # create MHN for predictable and unpredictable elements
    net = ContinuousHopfield(np.prod(dims) + latent_dim, beta=beta)
    # get elements where error is above threshold and remap to range [-1,1]
    # first compute the average error across the three channels
    avg_diff = np.mean(abs(diff), axis=-1, keepdims=True)
    # next create a mask where the average error is greater than the threshold
    mask = avg_diff > threshold
    # finally create the output image using the mask and input images
    sparse_to_encode = np.where(mask, (test_data_squares * 2) - 1, 0)

    # get latent vectors for test memories
    _, latents = get_recalled_ims_and_latents(vae, test_data_squares, noise_level=0)

    # NEW CODE
    ims_with_diff_shapes = [get_diff_shape_im(df, i, shape_id) for i in range(4)]
    recons, pred_labels = get_recalled_ims_and_latents(vae, np.array(ims_with_diff_shapes), noise_level=0)
    pred_labels = np.array([pred_labels[0][i] for i in range(len(pred_labels[0]))])
    pred_labels = pred_labels.reshape((4, 20, 1))
    fixed_latents = pred_labels
       
    # encode traces in MHN
    a = sparse_to_encode.reshape((4, np.prod(dims), 1))
    print(a.shape)
    print(pred_labels.shape)
    hpc_traces = np.concatenate((a, pred_labels), axis=1)
    net.learn(hpc_traces[0:4])

    # now visualise recall from a noisy image in the MHN
    images_masked_np = noise(hpc_traces, noise_factor=0.3, gaussian=True)

    tests = []
    label_inputs = []
    predictions = []
    pred_labels = []

    for test_ind in range(4):
        test_in = images_masked_np[test_ind].reshape(-1, 1)
        test_out = net.retrieve(test_in, max_iter=5)
        reconstructed = test_out[0:np.prod(dims)]
        input_im = test_in[0:np.prod(dims)]

        predictions.append(np.array(reconstructed).reshape((1, dims[0], dims[1], dims[2])))
        tests.append(np.array(input_im).reshape((1, dims[0], dims[1], dims[2])))
        pred_labels.append(test_out[np.prod(dims):])
        label_inputs.append(test_in[np.prod(dims):])

    predictions = np.concatenate(predictions, axis=0)
    tests = np.concatenate(tests, axis=0)
    pred_labels = np.array(pred_labels).reshape(4, latent_dim)
    label_inputs = np.array(label_inputs).reshape(4, latent_dim)

    # for the first ten test images, display the stages of recall
    final_outputs = []
    for test_ind in range(4):
        fig, final_im = recall_memories(test_data_squares, net, vae, dims, latent_dim, test_ind=test_ind,
                                        noise_factor=0.1, threshold=threshold, return_final_im=True,
                                        fixed_latents=fixed_latents[test_ind])


In [ ]:
test_extended_model_carmichael(filtered_df, 
                               dataset='shapes3d', 
                               vae=vae, 
                               latent_dim=20,
                               threshold = 0.2,
                               shape_id=0)
test_extended_model_carmichael(filtered_df, 
                               dataset='shapes3d',
                               vae=vae, 
                               latent_dim=20,
                               threshold = 0.2,
                               shape_id=2)